In [1]:
include("../../L-BFGS/OracleFunction.jl")
include("../../L-BFGS/BFGS.jl")
include("../../utilities/genFunc.jl")
using .BFGS
using .OracleFunction
using LinearAlgebra, BenchmarkTools, CSV, DataFrames

baseDir = joinpath("../", "results/LBFGS/comparison_BFGS/")
mkpath(baseDir);

In [6]:
# parameters for exact LS vs AWLS vs quadratic function 
λ        = 10^1
ϵ        = 10^-14
maxIters = 1:200
m        = 1000
n        = 20
num_trials = 10

gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)
non_quadratic = OracleF(ones(size(gf[:X_hat], 2)),
        (x) -> norm(gf[:X_hat] * x - gf[:y_hat]),
        (x) -> inv(norm(gf[:X_hat] * x - gf[:y_hat])) * gf[:X_hat]' * (gf[:X_hat] * x - gf[:y_hat])
    )
quadratic = OracleF(ones(size(gf[:X_hat], 2)),
        (x) -> norm(gf[:X_hat] * x - gf[:y_hat]) ^ 2,
        (x) -> 2 * gf[:X_hat]' * (gf[:X_hat] * x - gf[:y_hat])
    )
ls = LeastSquaresF(gf)

LeastSquaresF{Float64, Main.OracleFunction.var"#f#1"{Matrix{Float64}, Vector{Float64}}, Main.OracleFunction.var"#df#2"{Matrix{Float64}, Vector{Float64}}}(OracleF{Float64, Main.OracleFunction.var"#f#1"{Matrix{Float64}, Vector{Float64}}, Main.OracleFunction.var"#df#2"{Matrix{Float64}, Vector{Float64}}}([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Main.OracleFunction.var"#f#1"{Matrix{Float64}, Vector{Float64}}([0.6269851049934334 -0.9953855131718308 … 0.26819190323331954 -0.6780465661809252; -0.9332327939739362 0.8084805224537295 … 0.28452273460460176 0.9029306010736042; … ; 0.0 0.0 … 10.0 0.0; 0.0 0.0 … 0.0 10.0], [-10.186364190714537, -1.6231132115458327, 15.375526057866429, 1.5006156140417093, -14.546392908739245, -4.939853634188801, -0.2053418568399169, 20.47017872706546, 1.0276498922046493, -12.900436004701787  …  3.3479087325325874, 4.529977532247315, 9.395681468606902, -9.098074604739619, -8.545976276298948, 5.191283275708

In [7]:
### residual, relative error and norm of gradient with respect to iterations with exact line search
using Statistics, CSV

# Preallocate arrays
num_iterations = length(maxIters)

gradients = zeros(num_trials, num_iterations)
residuals = zeros(num_trials, num_iterations)
relative_errors = zeros(num_trials, num_iterations)

for trial in 1:num_trials

    gf = genFunc(:exactRandDataset, λ=λ, m=m, n=n)
    ls = LeastSquaresF(gf)

    for (i, maxIter) in enumerate(maxIters)
        t = BroydenFletcherGoldfarbShanno(ls, ϵ=ϵ, MaxEvaluations=maxIter)

        relative_errors[trial, i] = norm(t[:x] - gf[:w_star]) / norm(gf[:w_star])
        residuals[trial, i] = norm(gf[:X_hat] * t[:x] - gf[:y_hat]) / norm(gf[:y_hat])
        gradients[trial, i] = norm(t[:grad])
    end

    println("Done trial ", trial)
    
end

# Calculate mean and standard deviation
mean_relative = mean(relative_errors, dims=1)'
std_relative = std(relative_errors, dims=1)'
mean_residual = mean(residuals, dims=1)'
std_residual = std(residuals, dims=1)'
mean_gradient = mean(gradients, dims=1)'
std_gradient = std(gradients, dims=1)'


# Write results to CSV
outputvsc = joinpath(baseDir, "statisticsBFGS-iterations-m" * string(m) * "n" * string(n) * "--error-norm.csv");

accData = Dict(
    :maxiterations => Array{Int64}(undef, 0),
    :mean_relative => Array{Float64}(undef, 0),
    :std_relative  => Array{Float64}(undef, 0),
    :mean_residual => Array{Float64}(undef, 0),
    :std_residual  => Array{Float64}(undef, 0),
    :mean_gradient => Array{Float64}(undef, 0),
    :std_gradient  => Array{Float64}(undef, 0)
)

# create dataframe with columns from arrays
for maxIter ∈ maxIters
    push!(accData[:maxiterations], maxIter)
    push!(accData[:mean_relative], mean_relative[maxIter])
    push!(accData[:std_relative], std_relative[maxIter])
    push!(accData[:mean_residual], mean_residual[maxIter])
    push!(accData[:std_residual], std_residual[maxIter])
    push!(accData[:mean_gradient], mean_gradient[maxIter])
    push!(accData[:std_gradient], std_gradient[maxIter])

end


CSV.write(outputvsc, accData)


Done trial 1
Done trial 2
Done trial 3
Done trial 4
Done trial 5
Done trial 6
Done trial 7
Done trial 8
Done trial 9
Done trial 10


"../results/LBFGS/comparison_BFGS/statisticsBFGS-iterations-m1000n20--error-norm.csv"